## 1. Imports and data load

In [1]:
import pandas as pd
import nltk
import string
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk import word_tokenize
from gensim.models import Word2Vec as w2v
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
import re
import tensorflow as tf
from tensorflow import keras
import string
from nltk.stem import PorterStemmer

from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
import numpy as np
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import roc_auc_score, classification_report

2023-01-25 19:09:39.586922: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('Cleaned_Data.csv')

In [3]:
df

,Unnamed: 0,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,Number_of_tags,length_of_title,length_of_abstract,TITLE_modified,ABSTRACT_modified
0,0,1,Reconstructing Subject Specific Effect Maps,Predictive models allow subject specific infe...,1,0,0,0,0,0,1,4,265,reconstruct subject specif effect map,predict model allow subject specif infer anal...
1,1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarianc...,1,0,0,0,0,0,1,4,76,rotat invari neural network,rotat invari translat invari great valu imag ...
2,2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spheri...,0,0,1,0,0,0,1,8,99,spheric polyharmon poisson kernel polyharmon f...,introduc develop notion spheric polyharmon na...
3,3,4,A finite element approximation for the stochas...,The stochastic Landau Lifshitz Gilbert LLG eq...,0,0,1,0,0,0,1,9,110,finit element approxim stochast maxwel landau ...,stochast landau lifshitz gilbert llg equat co...
4,5,6,On maximizing the fundamental frequency of the...,Let $\Omega \subset \mathbb R ^n$ be a bounde...,0,0,1,0,0,0,1,11,210,maxim fundament frequenc complement obstacl,let omega subset mathbb r n bound domain sati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20238,20967,20968,Contemporary machine learning a guide for prac...,Machine learning is finding increasingly broa...,1,1,0,0,0,0,2,11,194,contemporari machin learn guid practition phys...,machin learn find increasingli broad applic p...
20239,20968,20969,Uniform diamond coatings on WC Co hard alloy c...,Polycrystalline diamond coatings have been gr...,0,1,0,0,0,0,1,15,134,uniform diamond coat wc co hard alloy cut inse...,polycrystallin diamond coat grown cement carb...
20240,20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying sit...,1,0,0,0,0,0,1,7,194,analys soccer game cluster conceptor,present new approach identifi situat behaviou...
20241,20970,20971,On the Efficient Simulation of the Left Tail o...,The sum of Log normal variates is encountered...,0,0,1,1,0,0,2,14,193,effici simul left tail sum correl log normal v...,sum log normal variat encount mani challeng a...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20243 entries, 0 to 20242
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            20243 non-null  int64 
 1   ID                    20243 non-null  int64 
 2   TITLE                 20243 non-null  object
 3   ABSTRACT              20243 non-null  object
 4   Computer Science      20243 non-null  int64 
 5   Physics               20243 non-null  int64 
 6   Mathematics           20243 non-null  int64 
 7   Statistics            20243 non-null  int64 
 8   Quantitative Biology  20243 non-null  int64 
 9   Quantitative Finance  20243 non-null  int64 
 10  Number_of_tags        20243 non-null  int64 
 11  length_of_title       20243 non-null  int64 
 12  length_of_abstract    20243 non-null  int64 
 13  TITLE_modified        20243 non-null  object
 14  ABSTRACT_modified     20243 non-null  object
dtypes: int64(11), object(4)
memory usage

In [5]:
drop_index = df[(df['Quantitative Biology'] == 1)
                |(df['Quantitative Finance'] == 1)].index
df.drop(drop_index, inplace = True)

In [6]:
df

,Unnamed: 0,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,Number_of_tags,length_of_title,length_of_abstract,TITLE_modified,ABSTRACT_modified
0,0,1,Reconstructing Subject Specific Effect Maps,Predictive models allow subject specific infe...,1,0,0,0,0,0,1,4,265,reconstruct subject specif effect map,predict model allow subject specif infer anal...
1,1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarianc...,1,0,0,0,0,0,1,4,76,rotat invari neural network,rotat invari translat invari great valu imag ...
2,2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spheri...,0,0,1,0,0,0,1,8,99,spheric polyharmon poisson kernel polyharmon f...,introduc develop notion spheric polyharmon na...
3,3,4,A finite element approximation for the stochas...,The stochastic Landau Lifshitz Gilbert LLG eq...,0,0,1,0,0,0,1,9,110,finit element approxim stochast maxwel landau ...,stochast landau lifshitz gilbert llg equat co...
4,5,6,On maximizing the fundamental frequency of the...,Let $\Omega \subset \mathbb R ^n$ be a bounde...,0,0,1,0,0,0,1,11,210,maxim fundament frequenc complement obstacl,let omega subset mathbb r n bound domain sati...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20238,20967,20968,Contemporary machine learning a guide for prac...,Machine learning is finding increasingly broa...,1,1,0,0,0,0,2,11,194,contemporari machin learn guid practition phys...,machin learn find increasingli broad applic p...
20239,20968,20969,Uniform diamond coatings on WC Co hard alloy c...,Polycrystalline diamond coatings have been gr...,0,1,0,0,0,0,1,15,134,uniform diamond coat wc co hard alloy cut inse...,polycrystallin diamond coat grown cement carb...
20240,20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying sit...,1,0,0,0,0,0,1,7,194,analys soccer game cluster conceptor,present new approach identifi situat behaviou...
20241,20970,20971,On the Efficient Simulation of the Left Tail o...,The sum of Log normal variates is encountered...,0,0,1,1,0,0,2,14,193,effici simul left tail sum correl log normal v...,sum log normal variat encount mani challeng a...


## 2. Data processing

In [7]:
#X = df[['TITLE', 'ABSTRACT', 'Number_of_tags', 'length_of_title', 
#'length_of_abstract']]
X = df[['TITLE', 'ABSTRACT', 'TITLE_modified', 'ABSTRACT_modified',
        'length_of_title', 'length_of_abstract']]

y = df[['Computer Science', 'Physics', 'Mathematics', 'Statistics']]

In [8]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size = 0.2, random_state = 42)

In [10]:
def vectorize_fit(function, data):
    tf1 = function
    tf1.fit(data['TITLE_modified'])
    tf2 = function
    tf2.fit(data['ABSTRACT_modified'])
    return (tf1, tf2)

In [11]:
def vectorize_transform(function, data):
    data1 = function[0].transform(data['TITLE_modified'])
    data2 = function[1].transform(data['ABSTRACT_modified'])
    data = tf.concat([tf.convert_to_tensor(data1.toarray()),
                 tf.convert_to_tensor(data2.toarray())], axis = 1)
    return data

In [12]:
function = vectorize_fit(CountVectorizer(), X_train[['TITLE_modified'
                                                     , 'ABSTRACT_modified']])

In [13]:
data_train_text = vectorize_transform(function, X_train[['TITLE_modified', 'ABSTRACT_modified']])
data_val_text = vectorize_transform(function, X_val[['TITLE_modified', 'ABSTRACT_modified']])
data_test_text = vectorize_transform(function, X_test[['TITLE_modified', 'ABSTRACT_modified']])

2023-01-25 19:09:53.340570: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
#data_train_num = scale.fit_transform(X_train[['Number_of_tags',
                                     #          'length_of_abstract', 
                                      #        'length_of_title']])
#data_val_num = scale.transform(X_val[['Number_of_tags', 
                                     #     'length_of_abstract',
                                    #      'length_of_title']])
#data_test_num = scale.transform(X_test[['Number_of_tags', 
                                     #     'length_of_abstract',
                                     #     'length_of_title']])

data_train_num = scale.fit_transform(X_train[['length_of_abstract', 
                                              'length_of_title']])
data_val_num = scale.transform(X_val[['length_of_abstract',
                                          'length_of_title']])
data_test_num = scale.transform(X_test[['length_of_abstract',
                                          'length_of_title']])

In [15]:
data_train_num = tf.convert_to_tensor(data_train_num)
data_val_num = tf.convert_to_tensor(data_val_num)
data_test_num = tf.convert_to_tensor(data_test_num)

In [16]:
data_train = [data_train_text, data_train_num]
data_val = [data_val_text, data_val_num]
data_test = [data_test_text, data_test_num]

## 3. Model

## 3.1 Model training

In [17]:
def create_model():
    input1 = tf.keras.Input(shape=(data_train[0].shape[1]))
    t1 = tf.keras.layers.Dense(200, activation='relu')(input1)
    t1 = tf.keras.layers.Dense(100, activation='relu')(t1)
    t1 = tf.keras.layers.Dense(50, activation='sigmoid')(t1)
    input2 = keras.Input(shape=(data_train[1].shape[1]))
    x = tf.concat([t1, input2],-1)
    output = keras.layers.Dense(4, activation='sigmoid')(x)
    model = keras.Model(inputs=[input1, input2], outputs=output)
    return model
    


In [18]:
def model_fit(model, data_train, target_train, data_val, target_val):
    model.compile(loss='binary_crossentropy',
       optimizer='sgd',
       metrics=['binary_accuracy'])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                 patience=5)
    history = model.fit([data_train[0], data_train[1]], 
                      target_train, epochs=80,
           validation_data=([data_val[0], data_val[1]],
                            target_val), callbacks=[early_stopping])
    
    
    

In [19]:
def model_predict(model, data):
    y_proba_pred = model.predict([data[0], data[1]])
    y_pred = (y_proba_pred >= np.array([0.5, 0.5, 0.5, 0.5]))
    return (y_proba_pred, y_pred) 

In [20]:
model = create_model()

In [21]:
model_fit(model, data_train, y_train, data_val, y_val)

Epoch 1/80
389/389 [==============================] - 22s 56ms/step - loss: 0.6466 - binary_accuracy: 0.6326 - val_loss: 0.6090 - val_binary_accuracy: 0.6818
Epoch 2/80
389/389 [==============================] - 22s 56ms/step - loss: 0.5995 - binary_accuracy: 0.6848 - val_loss: 0.5869 - val_binary_accuracy: 0.6952
Epoch 3/80
389/389 [==============================] - 27s 71ms/step - loss: 0.5766 - binary_accuracy: 0.6976 - val_loss: 0.5638 - val_binary_accuracy: 0.7122
Epoch 4/80
389/389 [==============================] - 23s 60ms/step - loss: 0.5507 - binary_accuracy: 0.7179 - val_loss: 0.5365 - val_binary_accuracy: 0.7262
Epoch 5/80
389/389 [==============================] - 25s 63ms/step - loss: 0.5205 - binary_accuracy: 0.7392 - val_loss: 0.5052 - val_binary_accuracy: 0.7518
Epoch 6/80
389/389 [==============================] - 24s 63ms/step - loss: 0.4871 - binary_accuracy: 0.7680 - val_loss: 0.4721 - val_binary_accuracy: 0.7834
Epoch 7/80
389/389 [==============================] 

## 3.2 Model prediction on the test set

In [22]:
y_test_proba_pred, y_test_pred = model_predict(model, data_test)

122/122 [==============================] - 4s 31ms/step


In [23]:
df_test_target_pred = pd.DataFrame(y_test_pred, columns = ['Computer Science', 
                                               'Physics', 'Mathematics',
                                              'Statistics'])

In [24]:
df_test_pred = pd.concat([X_test[['TITLE', 'ABSTRACT']].reset_index(), 
                          df_test_target_pred], 
                         axis = 1)

In [25]:
print('Predict tags on papers of the test sets:\n')
df_test_pred

Predict tags on papers of the test sets:



,index,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics
0,15978,Concurrent Segmentation and Localization for T...,Real time instrument tracking is a crucial re...,True,False,False,True
1,17315,Cascaded Incremental Nonlinear Dynamic Inversi...,Micro Aerial Vehicles MAVs are limited in the...,True,False,False,False
2,7534,The energy momentum tensor of electromagnetic ...,We present a complete resolution of the Abrah...,False,True,False,False
3,14535,Analytic continuation of Wolynes theory into t...,The Wolynes theory of electronically nonadiab...,False,True,False,False
4,10868,Query Efficient Black box Adversarial Examples...,"Note that this paper is superceded by ""Black ...",True,False,False,False
...,...,...,...,...,...,...,...
3884,17487,An Empirical Study of Mini Batch Creation Stra...,Training of neural machine translation NMT mo...,True,False,False,False
3885,99,Quasi two dimensional Fermi surface topography...,We report on a combined study of the de Haas ...,False,True,False,False
3886,3689,Two provably consistent divide and conquer clu...,In this article we advance divide and conquer...,True,False,False,True
3887,9146,Large Sample Asymptotics of the Pseudo Margina...,The pseudo marginal algorithm is a variant of...,False,False,True,True


In [26]:
df_test_target_proba_pred = pd.DataFrame(y_test_proba_pred, 
                                         columns = ['Computer Science', 
                                               'Physics', 'Mathematics',
                                              'Statistics'])

In [27]:
df_test_proba_pred = pd.concat([X_test[['TITLE', 'ABSTRACT']].reset_index(), 
                                df_test_target_proba_pred], 
                         axis = 1)

In [28]:
print('Predict tag probability on papers of the test sets:\n')
df_test_proba_pred

Predict tag probability on papers of the test sets:



,index,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics
0,15978,Concurrent Segmentation and Localization for T...,Real time instrument tracking is a crucial re...,0.896429,0.011701,0.013086,0.527511
1,17315,Cascaded Incremental Nonlinear Dynamic Inversi...,Micro Aerial Vehicles MAVs are limited in the...,0.661068,0.163869,0.302951,0.002308
2,7534,The energy momentum tensor of electromagnetic ...,We present a complete resolution of the Abrah...,0.007013,0.998491,0.004217,0.001051
3,14535,Analytic continuation of Wolynes theory into t...,The Wolynes theory of electronically nonadiab...,0.004369,0.991787,0.063744,0.002285
4,10868,Query Efficient Black box Adversarial Examples...,"Note that this paper is superceded by ""Black ...",0.980559,0.003404,0.015650,0.257105
...,...,...,...,...,...,...,...
3884,17487,An Empirical Study of Mini Batch Creation Stra...,Training of neural machine translation NMT mo...,0.921358,0.028754,0.005103,0.321339
3885,99,Quasi two dimensional Fermi surface topography...,We report on a combined study of the de Haas ...,0.017885,0.983432,0.034108,0.003939
3886,3689,Two provably consistent divide and conquer clu...,In this article we advance divide and conquer...,0.880557,0.005279,0.109164,0.584815
3887,9146,Large Sample Asymptotics of the Pseudo Margina...,The pseudo marginal algorithm is a variant of...,0.084300,0.005899,0.664246,0.977860


## 4. Model performance evaluation

## 4.1 Performance on Computer Science subject

In [29]:
print('The AUC socre:', 
      roc_auc_score(y_test['Computer Science'], 
                    df_test_proba_pred['Computer Science']))

The AUC socre: 0.9331605786714148


In [30]:
print('Classification report:\n', 
      classification_report(y_test['Computer Science'], df_test_pred['Computer Science']))

Classification report:
               precision    recall  f1-score   support

           0       0.89      0.87      0.88      2223
           1       0.83      0.86      0.85      1666

    accuracy                           0.87      3889
   macro avg       0.86      0.87      0.86      3889
weighted avg       0.87      0.87      0.87      3889



## 4.2 Performance on Physics subject

In [31]:
print('The AUC socre:', 
      roc_auc_score(y_test['Physics'], df_test_proba_pred['Physics']))

The AUC socre: 0.9716647232277039


In [32]:
print('Classification report:\n', 
      classification_report(y_test['Physics'], df_test_pred['Physics']))

Classification report:
               precision    recall  f1-score   support

           0       0.94      0.97      0.96      2733
           1       0.93      0.86      0.90      1156

    accuracy                           0.94      3889
   macro avg       0.94      0.92      0.93      3889
weighted avg       0.94      0.94      0.94      3889



## 4.3 Performance on Mathematics subject 

In [33]:
print('The AUC socre:', 
      roc_auc_score(y_test['Mathematics'], 
                    df_test_proba_pred['Mathematics']))

The AUC socre: 0.9503322653376053


In [34]:
print('Classification report:\n', 
      classification_report(y_test['Mathematics'], 
                            df_test_pred['Mathematics']))

Classification report:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93      2809
           1       0.84      0.79      0.82      1080

    accuracy                           0.90      3889
   macro avg       0.88      0.87      0.87      3889
weighted avg       0.90      0.90      0.90      3889



## 4.4 Performance on Statistics subject

In [35]:
print('The AUC socre:', 
      roc_auc_score(y_test['Statistics'], 
                    df_test_proba_pred['Statistics']))

The AUC socre: 0.9363249453232647


In [36]:
print('Classification report:\n', 
      classification_report(y_test['Statistics'], 
                            df_test_pred['Statistics']))

Classification report:
               precision    recall  f1-score   support

           0       0.91      0.93      0.92      2921
           1       0.77      0.74      0.75       968

    accuracy                           0.88      3889
   macro avg       0.84      0.83      0.84      3889
weighted avg       0.88      0.88      0.88      3889

